**Change these 2 parameters only**

In [1]:
dataset_name = "train-v2.0"
split_size = 20 # train = 20, dev = 2

Run cells one by one and it will produce the translated JSON

In [2]:
import json
import numpy as np
import pandas as pd

In [3]:
df = pd.read_json(f'..\\data\\{dataset_name}.json')
del df['version']
df = np.array_split(df, split_size)

Parse JSONs to txt files for uploading to eTranslation

- wait for translated files and put them into the `/data/for_translating_purposes` directory

In [10]:
for i in range(split_size):
    with open(f"..\\data\\for_translating_purposes\\{dataset_name}-part{i+1}_EN.txt", "w", encoding="utf-8") as file:
        for _, dset in df[i].iterrows():
            title = dset['data']['title']
            file.write("[title:]\n")
            file.write(f"{title}\n")

            for row in dset['data']['paragraphs']:
                article_text = row['context']
                file.write("[context:]\n")
                file.write(f"{article_text}\n")

                for qas in row['qas']:
                    question_text = qas['question']
                    file.write("[question:]\n")
                    file.write(f"{question_text}\n")

                    if 'answers' in qas.keys():
                        for ans in qas['answers']:
                            answer_text = ans['text']
                            file.write("[answer:]\n")
                            file.write(f"{answer_text}\n")
                    
                    else:
                        for ans in qas['plausible_answers']:
                            answer_text = ans['text']
                            file.write("[answer:]\n")
                            file.write(f"{answer_text}\n")

Parse translated txt files back to JSON

In [12]:
df = pd.read_json(f'..\\data\\{dataset_name}.json')

INITIAL_VALUE = -1
new_data = list()
count = INITIAL_VALUE
paragraph_count = INITIAL_VALUE
question_count = INITIAL_VALUE
for i in range(split_size):

    with open(f"..\\data\\for_translating_purposes\\{dataset_name}-part{i+1}_EN_SL.txt", "r", encoding="utf-8") as file:
        file_lines = file.readlines()

        for i in range(len(file_lines)):
            line = file_lines[i]

            if "[naslov:]" in line:
                paragraph_count = INITIAL_VALUE
                count += 1
                next_line = file_lines[i + 1].rstrip("\n")
                new_data.append({'title': next_line, 'paragraphs': []})

            if "[kontekst:]" in line:
                question_count = INITIAL_VALUE
                paragraph_count += 1
                next_line = file_lines[i + 1].rstrip("\n")
                new_data[count]['paragraphs'].append({'qas': [], 'context': next_line})

            if "[vprašanje:]" in line:
                question_count += 1
                next_line = file_lines[i + 1].rstrip("\n")
                new_data[count]['paragraphs'][paragraph_count]['qas'].append({'question': next_line, 'answers': []})

            if "[odgovor:]" in line:
                next_line = file_lines[i + 1].rstrip("\n")
                new_data[count]['paragraphs'][paragraph_count]['qas'][question_count]['answers'].append({'text': next_line})

# add -1 as indices for answer start
for article in new_data:
    for paragraph in article['paragraphs']:
        for qas in paragraph['qas']:
            if len(qas['answers']) == 0:
                qas['is_impossible'] = True
            else:
                qas['is_impossible'] = False
                for ans in qas['answers']:
                    ans['answer_start'] = -1
                    ans['answer_end'] = -1


with open(f"..\\data\\{dataset_name}_unaligned_SL.json", "w", encoding="utf-8") as new_file:
    d = {'data': new_data}
    new_file.write(json.dumps(d, indent=2))

# add indices of answer start from english version
for i, dset in df.iterrows():
    article = dset['data']
    for p, paragraph in enumerate(article['paragraphs']):
        for q, qas in enumerate(paragraph['qas']):
            for a, ans in enumerate(qas['answers']):
                new_data[i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start'] = ans['answer_start']
                new_data[i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_end'] = ans['answer_start'] + len(new_data[i]['paragraphs'][p]['qas'][q]['answers'][a]['text'])

with open(f"..\\data\\{dataset_name}_unaligned_original_indices_SL.json", "w", encoding="utf-8") as new_file:
    d = {'data': new_data}
    new_file.write(json.dumps(d, indent=2))